# Mastering with ROS: Turtlebot3

<img src="img/robotis_logo.png" width="400" />

<img src="img/blob_unit.png" width="600" />

<img src="img/robotignite_logo_text.png" width="400"/>

# Unit 4: Blob Tracking

<p style="background:green;color:white;">SUMMARY</p>

Estimated time of completion: **2h**

In this unit, you will start using cameras in ROS and use the cmvision package for blob tracking. Once you get the hang of it, then in Unit 2, you will go deeper in how this blob tracking is done and how the image can be processed.

<p style="background:green;color:white;">END OF SUMMARY</p>

Now, you are going to see what the robot Turtlebot3 is seeing. For that, you are going to use a ROS graphical tool called <i>rqt_image_view</i> that allows you to see what the camera in the robot is publishing.

To open the tool, type the following:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rosrun rqt_image_view rqt_image_view

Now, open the Graphical Display by clicking on the icon:

<img src="img/font-awesome_desktop.png">

Then, you should see on the screen a window of the **rqt_image_view** application.
 
On the application, select the **/camera/rgb/image_raw** image topic and wait a few seconds until the image feed is established. You should see something similar to this.

<img src="img/rqt_image.png" width="600" />

<p style="background:#EE9023;color:white;">**Exercise 4.1**</p>

Great! Now get move Turtlebot3 around until the red ball appears in the robot's view. Once it is in view of the Turtlebot3 robot, you can close the program of WebShell #2.

<p style="background:#EE9023;color:white;">**END Exercise 4.1**</p>

## Blob tracking with OpenCV and Python  part 1, color encoding

Now, what you are going to do is create a program that can track blobs of colour in an image.<br>
Blobs are nothing more than areas in an image with a similar color encoding, as similar as you might define it.<br>
So, obviously, the first step is to get a color encoding that defines the object you want to be tracked.<br>
Let's do that with the red ball, shall we?<br>

To obtain the color encoding, we are going to use another tool provided by ROS that allows to easily compute the RBG and YUV values of a target blob.
 
Launch the following command in a terminal and go to the Graphical Interface Tab:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
roslaunch blob_tracking_t3 start_colour_gui.launch

This will start a gui similar to the following:

<img src="img/color_gui_unfocus.png" width="500" />

Unzoom the image a little bit in order to be able to visualize the ball.

<img src="img/color_gui1.png" width="500" />

Now, click over the red ball and see how by successively clicking over slightly different points of the ball, a black box appears around it. Click over it until the box is just over the entire red ball:

<img src="img/color_gui2.png" width="500" />

See the two text boxes? You have the RGB value of the color on average and then you have the YUV, which, in this case, is **(30:82, 86:111, 178:252)**. Yours could be slightly different. But, the point is that this YUV defines the color blob to be tracked. This means that this is what Turtlebot3 will consider to be the RedBall.

You can now close the WebShell #2 program.

#### We are going to use those values to create a configuration file, required by the blob recognition code.

Now, let's create a package to launch all of the required software to track the ball.

* First, create a new package named <i>my_blob_tracking_pkg</i>, which depends on <i>rospy</i>.
* Inside that package, create a directory named <i>color_files</i> and put the <i>colors.txt</i> file in it. Add the values you got in the previous step, by writing the RGB in average and the YUV values like in the file below.


<p style="background:green;color:white;">**colors.txt**</p>

In [ ]:
[Colors]
(  0, 255,  0) 0.000000 10 Green
(  0,  0, 255) 0.000000 10 Blue
(  255,  255, 0) 0.000000 10 Yellow
(  255,  0, 255) 0.000000 10 Purple
(  0,  255, 255) 0.000000 10 Teal
(  254,  4,   4) 0.000000 10 RedBall

[Thresholds]
( 144:154, 38:48, 16:26 )
( 24:34, 250:255, 102:112 )
( 220:230, 0:5, 143:153 )
( 100:110, 207:217, 229:239)
( 173:183, 165:175, 0:5 )
( 30:81, 86:111, 178:253 )

<p style="background:green;color:white;">**END colors.txt**</p>

You see that there are two parts: **Colors** and **Thresholds**<br>
In **Colors**, you state:<br>

* The RGB value of the line around the detection, in this case, is the same color as the average RGB color detected previously, but you can put any color you want. It's just good practice to put the Average color because, that way, you can see what blob is being detected.
* The other two numbers are not used in this version of cmvision.
* The name of the blob representation, in this case RedBall


In **Thresholds**, you state:<br>

* Essentially, the Red, Green, and Blue value range. In the case of the RedBall, Red = [from 30 to 81], Green = [from 86 to 111], and Blue = [from 178 to 253]
* All of them are placed in the same order as the Color List to make the correspondence.


As you can see, you can put as many colors as you wish and, afterwards, be able to track different blobs based on their names, like Green, RedBall, Teal, etc.

## Blob tracking with OpenCV and Python part 2, Start blob tracking with cmvision

The ROS package that you have used for the color setting and that you will use for the tracking is **cmvision**, http://wiki.ros.org/cmvision.

Add the following launch file that starts the blob tracking code based on your <i>colour.txt</i> file, connect to Turtlebot3's camera, and publish the ball position and information in a ROS topic. You must add this launch file to the launch directory of your package.


<p style="background:green;color:white;">**my_t3_cmvision_tc.launch**</p>

In [ ]:
<launch>
  
  <arg name="rgb_raw_image_topic" default="/camera/rgb/image_raw"/>
  <arg name="color_file_path" default="$(find my_blob_tracking_pkg)/color_files/colors.txt"/>
  
  <!-- Location of the cmvision color file -->
  <param name="cmvision/color_file" type="string" 
         value="$(arg color_file_path)" />

  <!-- Turn debug output on or off -->
  <param name="cmvision/debug_on" type="bool" value="true"/>

  <!-- Turn color calibration on or off -->
  <param name="cmvision/color_cal_on" type="bool" value="false"/>

  <!-- Enable Mean shift filtering -->
  <param name="cmvision/mean_shift_on" type="bool" value="false"/>

  <!-- Spatial bandwidth: Bigger = smoother image -->
  <param name="cmvision/spatial_radius_pix" type="double" value="2.0"/>

  <!-- Color bandwidth: Bigger = smoother image-->
  <param name="cmvision/color_radius_pix" type="double" value="40.0"/>

  <node name="cmvision" pkg="cmvision" type="cmvision" args="image:=$(arg rgb_raw_image_topic)" 
        output="screen" />
</launch>


<p style="background:green;color:white;">**END my_t3_cmvision_tc.launch**</p>

Here, you set the image topic from which the RGB data will be extracted:

In [ ]:
<arg name="rgb_raw_image_topic" default="/camera/rgb/image_raw"/>

Here, you set the path to the color file you created through the given path or, if not, the default value:

In [ ]:
<arg name="color_file_path" default="$(find my_blob_tracking_pkg)/color_files/colors.txt"/>
<!-- Location of the cmvision color file -->
  <param name="cmvision/color_file" type="string" 
         value="$(arg color_file_path)" />

Here, you launch the cmvision node that will make the blob tracking happen based on the image topic given:

In [ ]:
<node name="cmvision" pkg="cmvision" type="cmvision" args="image:=$(arg rgb_raw_image_topic)" 
        output="screen" />

The other parameters are irrelevant for this course.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch my_blob_tracking_pkg my_t3_cmvision_tc.launch

NOTE: If you see the error <i>libdc1394 error: Failed to initialize libdc1394</i> do not worry. Everything is fine.

You should, then, have blob information published in the <i>/blobs</i> topic. Let's check what is published in that topic:


<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rostopic list | grep /blobs

That should show the topic <i>/blobs</i> on the screen.

Now, let's check some info about the topic:


<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rostopic info /blobs

It will give you the following output:

In [ ]:
user ~ $ rostopic info /blobs            
Type: cmvision/Blobs                      
Publishers:                              
 * /cmvision (http://ip-172-31-36-185:42984/)
Subscribers: None

Let's analize the message obtained through that topic:

In [ ]:
user ~ $ rosmsg show cmvision/Blobs                                   
std_msgs/Header header                                    
  uint32 seq                                    
  time stamp                                    
  string frame_id                                   
uint32 image_width                                    
uint32 image_height                                   
uint32 blob_count                                   
cmvision/Blob[] blobs                                   
  string name                                   
  uint32 red                                    
  uint32 green                                    
  uint32 blue                                   
  uint32 area                                   
  uint32 x                                    
  uint32 y                                    
  uint32 left                                   
  uint32 right                                    
  uint32 top                                    
  uint32 bottom

Basically, the <i>Blobs</i> message is an array of <i>cmVision/blob</i> messages.

So, what is in a <i>cmVision/blob</i> message?:

In [ ]:
user ~ $ rosmsg show cmvision/Blob                                    
string name                                   
uint32 red                                    
uint32 green                                    
uint32 blue                                   
uint32 area                                   
uint32 x                                    
uint32 y                                    
uint32 left                                   
uint32 right                                    
uint32 top                                    
uint32 bottom

Each <i>cmVision/blob</i> message provides the information for a single detected blob.

As you can see, you will get a lot of information out of each detected blob. Although the most important information is the blob name, the position in the image, and the size. With this, you can position the blob in a 2D, and maybe in a 3D, space and know which blob it is, able to track multiple objects in 3D space, if they are sufficiently different in colour.

And you should see the Turtlebot3 camera with the tracking in the Graphical Interface already working. If you don't have a red box around the ball, you should check that the YUV you gave are enough, and run the color setup again, if necessary.

<img src="img/perception_unit1_cmvisiontrack1.png">

<p style="background:#EE9023;color:white;">**Exercise 4.2**</p>

Make a python script that retrieves the needed information from the /blob topic to be able to:<br>

* Filter all the blobs, except the RedBall blobs
* Retrieve its position in 2D in the image
* Make Turtlebot3 move so that the red ball is in the center of the image all the time. 
* Move the ball using the keyboard and really check that the robot is moving to always keep the red ball in the middle of the screen. 

The result should be something like this:

<img src="img/blob_tracking.gif" width="600" />

In order to move the ball with the keyboard, you can use the following command:

In [ ]:
roslaunch blob_tracking move_ball_keyboard.launch

<p style="background:#EE9023;color:white;">**END Exercise 4.2**</p>

<p style="background:green;color:white;">**Solution Exercise 4.2**</p>

Please try to do it by yourself unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

Here you have an example of how it could be done:

In [ ]:
#!/usr/bin/env python
import rospy
from geometry_msgs.msg import Twist
from cmvision.msg import Blobs, Blob

#global
turn = 0.0 #turning rate
blob_position = 0 # x position for the blob

# callback function checks to see if any blobs were found then
# loop through each and get the x position.  Since the camera
# will sometimes find many blobs in the same object we just
# average all the x values.  You could also just take the first
# one if you are sure you will only have one blob. 
#
# This doesn't use multiple blobs but if are tracking several 
# objects you need to check the /data.blobs.color topic for
# the color tag you put in your colors.txt file. 
#
# after we have the x value we just make the robot turn to 
# keep it in the center of the image.

def callback(data):
    global turn
    global blob_position

    if(len(data.blobs)):

        for obj in data.blobs:
            if obj.name == "RedBall":
                rospy.loginfo("Blob <"+str(obj.name)+"> Detected!")
                blob_position = obj.x
        
                rospy.loginfo("blob is at %s"%blob_position)
                # turn right if we set off the left cliff sensor
                if( blob_position > 1000 ):
                    rospy.loginfo("TURN RIGHT")
                    turn = -1.0
                # turn left if we set off the right cliff sensor
                if( blob_position < 900 ):
                    rospy.loginfo("TURN LEFT")
                    turn = 1.0
        
                if( blob_position > 900 and blob_position < 1000):
                    rospy.loginfo("CENTERED")
                    turn = 0.0
    else: 
        turn = 0.0

def run():
    rospy.init_node("track_blob_color_node", log_level=rospy.WARN)
    global blob_position
    # publish twist messages to /cmd_vel
    pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)

    #subscribe to the robot sensor state
    rospy.Subscriber('/blobs', Blobs, callback)
    

    global turn
    twist = Twist()

    while not rospy.is_shutdown():

        # turn if we hit the line
        if ( turn != 0.0 ):
            str = "Turning %s"%turn
            rospy.loginfo(str)
            twist.linear.x = 0.0; twist.linear.y = 0; twist.linear.z = 0
            twist.angular.x = 0; twist.angular.y = 0; twist.angular.z = turn
            turn = 0.0

            # straight otherwise
        else:
            str = "Straight %s"%turn
            rospy.loginfo(str)
            twist.linear.x = 0.0; twist.linear.y = 0; twist.linear.z = 0
            twist.angular.x = 0; twist.angular.y = 0; twist.angular.z = 0

            # send the message and delay
        pub.publish(twist)
        blob_position = 0
        rospy.sleep(0.1)

if __name__ == '__main__':
    try:
        run()
    except rospy.ROSInterruptException: pass

## Congratulations! You can now track anything with color. 